In [8]:
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats.stats import pearsonr
from scipy.stats import norm

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

## Loading Data

In [9]:
with open('X_train.npy', 'rb') as f:
    X_train = np.load(f)
with open('X_test.npy', 'rb') as f:
    X_test = np.load(f)
with open('y_train.npy', 'rb') as f:
    y_train = np.load(f)
with open('y_test.npy', 'rb') as f:
    y_test = np.load(f)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(13972, 28) (4069, 28) (13972,) (4069,)


In [10]:
with open('X_train2.npy', 'rb') as f:
    X_train2 = np.load(f)
with open('X_test2.npy', 'rb') as f:
    X_test2 = np.load(f)
# with open('X_train3.npy', 'rb') as f:
#     X_train3 = np.load(f)
# with open('X_test3.npy', 'rb') as f:
#     X_test3 = np.load(f)

In [11]:
columns = ['Administrative',
 'Administrative_Duration',
 'Informational',
 'Informational_Duration',
 'ProductRelated',
 'ProductRelated_Duration',
 'BounceRates',
 'ExitRates',
 'PageValues',
 'SpecialDay',
 'Month_Aug',
 'Month_Dec',
 'Month_Feb',
 'Month_Jul',
 'Month_June',
 'Month_Mar',
 'Month_May',
 'Month_Nov',
 'Month_Oct',
 'Month_Sep',
 'VisitorType_New_Visitor',
 'VisitorType_Other',
 'VisitorType_Returning_Visitor',
 'Weekend_True',
 'OperatingSystems',
 'Browser',
 'Region',
 'TrafficType']

feature_names = columns

In [12]:
X_train = pd.DataFrame(X_train, columns=feature_names)
X_test = pd.DataFrame(X_test, columns=feature_names)

In [13]:
X_DF = pd.concat([X_train,X_test])
y = np.concatenate((y_train,y_test))

## Tuning Hyperparameters

In [14]:
model = AdaBoostClassifier()

In [15]:
param_dist = {
    "n_estimators":   list(np.linspace(200, 1600, 8, dtype=int)),
    "learning_rate": [0.15, 0.1, 0.05, 0.01, 0.005, 0.001]
}
# Random search of parameters, using cross validation, use all cores
#model_rand = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=100, cv=3, verbose=2, random_state=0, n_jobs=-1)

In [16]:
def get_best_params(model,param_dist,X,y):
    model_rand = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=100, cv=3, verbose=2, random_state=0, n_jobs=-1)
    model_rand.fit(X,y)
    best_params = model_rand.best_params_
    return best_params

In [17]:
best_params = get_best_params(model,param_dist,X_train,y_train)
print(best_params)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


/Users/erinszeto/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 48 is smaller than n_iter=100. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'n_estimators': 1600, 'learning_rate': 0.15}


## Fitting the Model

In [18]:
model_best = AdaBoostClassifier(**best_params)
model_best.fit(X_train,y_train) 

AdaBoostClassifier(learning_rate=0.15, n_estimators=1600)

In [19]:
score = cross_val_score(model_best, X_train, y_train)
print(np.round(score.mean(), 4))

0.8874


In [20]:
def get_performance(model,X,y):
    y_pred = model.predict(X)
    target_names = ['0: No Revenue', '1: Revenue']
    report = classification_report(y, y_pred, target_names=target_names)
    
    report_file = classification_report(y, y_pred, target_names=target_names, output_dict=True)
    
    print(report_file)
    print('\n')
    
    report_file = pd.DataFrame(report_file).transpose()
    report_file.to_csv(f"{str(model)}_report1.csv",index=False)
    
    print(report)
    print("Confusion Matrix:\n",confusion_matrix(y,y_pred))
    print("Accuracy: ",accuracy_score(y,y_pred))
    print("ROC-AUC: ",roc_auc_score(y,model.predict_proba(X)[:,1]))

In [21]:
get_performance(model_best,X_test,y_test)

{'0: No Revenue': {'precision': 0.94333936106088, 'recall': 0.9109429569266589, 'f1-score': 0.9268581581285165, 'support': 3436}, '1: Revenue': {'precision': 0.59254327563249, 'recall': 0.7030015797788309, 'f1-score': 0.6430635838150288, 'support': 633}, 'accuracy': 0.8785942492012779, 'macro avg': {'precision': 0.767941318346685, 'recall': 0.8069722683527449, 'f1-score': 0.7849608709717726, 'support': 4069}, 'weighted avg': {'precision': 0.8887672494668346, 'recall': 0.8785942492012779, 'f1-score': 0.882709235656057, 'support': 4069}}


               precision    recall  f1-score   support

0: No Revenue       0.94      0.91      0.93      3436
   1: Revenue       0.59      0.70      0.64       633

     accuracy                           0.88      4069
    macro avg       0.77      0.81      0.78      4069
 weighted avg       0.89      0.88      0.88      4069

Confusion Matrix:
 [[3130  306]
 [ 188  445]]
Accuracy:  0.8785942492012779
ROC-AUC:  0.9063215521189082


## Model with Feature Engineering

In [22]:
best_params = get_best_params(model,param_dist,X_train2,y_train)
print(best_params)

model2 = AdaBoostClassifier(**best_params)
model2.fit(X_train2, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


/Users/erinszeto/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 48 is smaller than n_iter=100. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'n_estimators': 1600, 'learning_rate': 0.15}


AdaBoostClassifier(learning_rate=0.15, n_estimators=1600)

In [23]:
get_performance(model2,X_test2,y_test)

{'0: No Revenue': {'precision': 0.9373515439429929, 'recall': 0.9188009313154831, 'f1-score': 0.9279835390946504, 'support': 3436}, '1: Revenue': {'precision': 0.6019971469329529, 'recall': 0.6666666666666666, 'f1-score': 0.6326836581709145, 'support': 633}, 'accuracy': 0.8795772917178668, 'macro avg': {'precision': 0.7696743454379729, 'recall': 0.7927337989910749, 'f1-score': 0.7803335986327824, 'support': 4069}, 'weighted avg': {'precision': 0.8851816414344269, 'recall': 0.8795772917178668, 'f1-score': 0.8820447765916459, 'support': 4069}}


               precision    recall  f1-score   support

0: No Revenue       0.94      0.92      0.93      3436
   1: Revenue       0.60      0.67      0.63       633

     accuracy                           0.88      4069
    macro avg       0.77      0.79      0.78      4069
 weighted avg       0.89      0.88      0.88      4069

Confusion Matrix:
 [[3157  279]
 [ 211  422]]
Accuracy:  0.8795772917178668
ROC-AUC:  0.9061732754387609


In [24]:
model2 = AdaBoostClassifier(**best_params)
model2.fit(X_train2, y_train)
get_performance(model2,X_test2,y_test)

{'0: No Revenue': {'precision': 0.9373515439429929, 'recall': 0.9188009313154831, 'f1-score': 0.9279835390946504, 'support': 3436}, '1: Revenue': {'precision': 0.6019971469329529, 'recall': 0.6666666666666666, 'f1-score': 0.6326836581709145, 'support': 633}, 'accuracy': 0.8795772917178668, 'macro avg': {'precision': 0.7696743454379729, 'recall': 0.7927337989910749, 'f1-score': 0.7803335986327824, 'support': 4069}, 'weighted avg': {'precision': 0.8851816414344269, 'recall': 0.8795772917178668, 'f1-score': 0.8820447765916459, 'support': 4069}}


               precision    recall  f1-score   support

0: No Revenue       0.94      0.92      0.93      3436
   1: Revenue       0.60      0.67      0.63       633

     accuracy                           0.88      4069
    macro avg       0.77      0.79      0.78      4069
 weighted avg       0.89      0.88      0.88      4069

Confusion Matrix:
 [[3157  279]
 [ 211  422]]
Accuracy:  0.8795772917178668
ROC-AUC:  0.9061732754387609


## Model with Feature Selection and Feature Engineering

In [25]:
with open('X_alt_train.npy', 'rb') as f:
     X_alt_train = np.load(f)
with open('X_alt_test.npy', 'rb') as f:
     X_alt_test = np.load(f)
        
to_keep = [17, 4, 20, 5, 3, 10, 13, 19, 21, 22, 23, 1, 2, 26, 18, 14, 6, 9, 27, 11]

def get_X(to_keep,X_train,X_test,X_alt_train,X_alt_test):
    X_train_ss = X_train.iloc[:,to_keep]
    X_test_ss = X_test.iloc[:,to_keep]
    
    X_train3 = np.hstack((X_train_ss.to_numpy(),X_alt_train))
    X_test3 = np.hstack((X_test_ss.to_numpy(),X_alt_test))
    
    return X_train3, X_test3

X_train3,X_test3 = get_X(to_keep,X_train,X_test,X_alt_train,X_alt_test)

np.save("X_trainAB.npy",X_train3)
np.save("X_testAB.npy",X_test3)

In [26]:
best_params = get_best_params(model,param_dist,X_train3,y_train)
print(best_params)

model3 = AdaBoostClassifier(**best_params)
model3.fit(X_train3, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


/Users/erinszeto/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 48 is smaller than n_iter=100. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'n_estimators': 1400, 'learning_rate': 0.1}


AdaBoostClassifier(learning_rate=0.1, n_estimators=1400)

In [27]:
get_performance(model3,X_test3,y_test)

{'0: No Revenue': {'precision': 0.9531875577456114, 'recall': 0.9007566938300349, 'f1-score': 0.926230734699985, 'support': 3436}, '1: Revenue': {'precision': 0.5851581508515815, 'recall': 0.7598736176935229, 'f1-score': 0.6611683848797251, 'support': 633}, 'accuracy': 0.8788400098304252, 'macro avg': {'precision': 0.7691728542985965, 'recall': 0.830315155761779, 'f1-score': 0.7936995597898551, 'support': 4069}, 'weighted avg': {'precision': 0.895934519022603, 'recall': 0.8788400098304252, 'f1-score': 0.8849959184217288, 'support': 4069}}


               precision    recall  f1-score   support

0: No Revenue       0.95      0.90      0.93      3436
   1: Revenue       0.59      0.76      0.66       633

     accuracy                           0.88      4069
    macro avg       0.77      0.83      0.79      4069
 weighted avg       0.90      0.88      0.88      4069

Confusion Matrix:
 [[3095  341]
 [ 152  481]]
Accuracy:  0.8788400098304252
ROC-AUC:  0.9095657079487335
